In [1]:
# Written 8/11/2022 to automate the filtering process and assign dam order to all dams in the nation.
# The starting file for this script is a csv file of the NID database dams for the entire nation 
# downloaded from https://nid.sec.usace.army.mil/#/downloads. You should be able to run this for only a subset
# of the dams as well.

# 1. Import databases and filter them. Combine databases as necessary.

In [2]:
# import packages
import pandas as pd
import numpy as np
import matplotlib as plt
import math
from simpledbf import Dbf5
import os

#import arcpy packages
import archook
arcgis_python_path = r'C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3' #The path on your computer where your ArcGIS Python installation is located

# Add the Python environment to the path
archook.arcpy = arcgis_python_path

# archook.get_arcpy()
import arcpy

pd.set_option('display.max_columns',None)

PyTables is not installed. No support for HDF output.


#### Load data here. You will need to replace all of the input files with the locations of your copies of the file. The output_folder variable is the folder where all of the output files will be saved to.

In [3]:
## load data. See the files included with the report for formatting.

# load NID data.
NIDs = pd.read_csv('D:/ResSed/MediumResolution_DamLinkages/Manuscript/FinalInputFiles/NID2024.csv',header=1)

#load removed dams file if using.
removed = pd.read_csv('D:/ResSed/MediumResolution_DamLinkages/Manuscript/FinalInputFiles/RemovedDams.csv')

#load spatial edits file if manually have changed any locations or deleted dams from NID.
editNID = pd.read_csv('D:/ResSed/MediumResolution_DamLinkages/Manuscript/FinalInputFiles/NID_spatialEdits.csv') #cross reference with deleting S001 etc.

#load GeoDAR data to use as location where no NID location.
geoDAR = pd.read_csv('D:/ResSed/MediumResolution_DamLinkages/Manuscript/FinalInputFiles/GeoDAR_crossref.csv')

#load Federal dam and site combined data for if you have sites with additional data that you want included. These dams
#have been manually placed on flowlines and have accurate locations.
sites = pd.read_csv('D:/ResSed/MediumResolution_DamLinkages/Manuscript/FinalInputFiles/sites.csv')

#NHD Plus Medium Resolution flowline shapefile
NHDFlowline = 'D:/ResSed/MediumResolution_DamLinkages/Manuscript/FinalInputFiles/NHDFlowline_Network_NHDPlus_Countries.gdb/NHDFlowline_Network_NHDPlus_Countries'  #path for NHD flowline shapefile

# load GRanD data. We are using v1.3 with some modifications to locations that places GRanD on NHD Flowlines.
GRanD = pd.read_csv('D:/ResSed/MediumResolution_DamLinkages/Manuscript/FinalInputFiles/GRanD_dams_v1_3.csv')

#D50 Data
D50 = pd.read_csv('D:/ResSed/MediumResolution_DamLinkages/Manuscript/FinalInputFiles/NHDPlus_MediumResolution_D50.csv',header=0)

#output folder directory where you want to save the output files.
out_folder = 'D:/ResSed/MediumResolution_DamLinkages/Manuscript/Outputs'


#Print original numbers of dams in various databases for tracking:
print('Original number of NID in database:',NIDs.shape)
print('Number of USBR in database:',sites.loc[sites.IsUSBR==1].shape)
print('Number of USACE in database:',sites.loc[sites.IsUSACE==1].shape)
print('Number of sites in database:',sites.loc[sites.IsSite==1].shape)

C:\Users\ahurst\AppData\Local\Temp\1\ipykernel_33772\1026226165.py:4: DtypeWarning: Columns (4,41,57,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  NIDs = pd.read_csv('D:/ResSed/MediumResolution_DamLinkages/Manuscript/FinalInputFiles/NID2024.csv',header=1)


Original number of NID in database: (91824, 84)
Number of USBR in database: (201, 32)
Number of USACE in database: (345, 32)
Number of sites in database: (1053, 32)


In [4]:
#Count how many dams have lat/long as 0.
noLoc = NIDs[(NIDs.Latitude == 0) | (NIDs.Longitude == 0)]

# noLoc.head()
print('Number of raw NIDs with no locations:', noLoc.shape)

Number of raw NIDs with no locations: (2, 84)


In [5]:
# Clean NID data to only keep the columns that we need
#FOR 2024 NID FILE. Column headers may change between versions of NID downloads. Check this if re-running with a new NID download.

#rename columns
new_cols = ['Dam_Name','Other_Dam','Former_Name','NID','OtherStructureID','FederalID','Owner_Name','OwnerTypes','PrimaryOwnerType','NumStruct','AssStruct','Designer','NonFedDam','PrimaryPurp', 'Purp','SourceAgency','StateorFedID','Latitude','Longitude','State','County','City','DisttoCity','River','CongressDist','AmInd','SecLoc','StateReg','Juris','Agency','StatePerm','StateInsp','StateEnforce','FedReg','FedOwner','FedFunding','FedDesign','FedConst','FedReg','FedInsp','FedOps','FedOther','SecAg','NRCS','PrimDamType','DamTypes','CoreTypes','Foundation','Dam_Height','HydraulicHeight','StructHeight','NID_Height','NIDHeightCat','Dam_Length','Volume','Year_Compl','YearCompCat','Year_Modif','NID_Storag','Max_Storag','Normal_Sto','Surface_Ar','Drainage_A','Max_Discha','SpillwayType','SpillWidth','NumLocks','LengthLocks','LockWidth','LengthSecondLock','SecondLockWidth','OutletGate','DataUpdated','LastInspection','InspectionFreq','HazardClass','CondAss','CondAssDate','OpStat','OpStatDate','EAPPrep','EAPRev','InundationMap','URL']
NIDs.columns = new_cols


# Filter and rename variables
NIDs = NIDs[['Dam_Name','Other_Dam','NID','OtherStructureID','FederalID','Longitude','Latitude','State','River','Owner_Name','Year_Compl','Year_Modif','NID_Height','Dam_Length','NID_Storag','Max_Storag','Normal_Sto','Surface_Ar','Drainage_A','Max_Discha','PrimaryPurp','Purp','PrimDamType','NumLocks','LengthLocks','LockWidth','LengthSecondLock','SecondLockWidth']]


#### Move and delete NID dams using the spatial edits file if so desired.

In [6]:
#Use spatial edit file to change lat/long of moved dams and delete deleted dams. If you do not wish to modify original NID, skip this.
delete = editNID.loc[editNID.Deleted == 1]
move = editNID.loc[editNID.Moved == 1]

NIDs = NIDs[~NIDs['NID'].isin(delete['NID_ID'])] #Drop delete NIDs

NIDs = pd.merge(NIDs, move, left_on = 'NID', right_on = 'NID_ID', how = 'left')

#Chane the Latitude and Longitude fields of the NIDs you wish to move.
NIDs.loc[NIDs.Moved == 1,'Latitude'] = NIDs.lat
NIDs.loc[NIDs.Moved == 1, 'Longitude'] = NIDs.long

print('Length to delete:', delete.shape)
print('NID database size:',NIDs.shape)


Length to delete: (91, 5)
NID database size: (91737, 33)


#### Join in the site file that has sites with additional data if using.

In [7]:
# Join the site data to the NID database

NID_join = pd.merge(NIDs, sites, on='NID', how='outer')

# Replace Storage NaNs with 0
NID_join.NID_Storag = NID_join.NID_Storag.fillna(0)
NID_join.Max_Storag = NID_join.Max_Storag.fillna(0)
NID_join.Normal_Sto = NID_join.Normal_Sto.fillna(0)
NID_join.IsUSBR = NID_join.IsUSBR.fillna(0)
NID_join.IsUSACE = NID_join.IsUSACE.fillna(0)
NID_join.IsSite = NID_join.IsSite.fillna(0)


#### Remove duplicate NIDs.

In [8]:
## Remove duplicate NIDs and keep the largest reported storage data
NIDs_join = NID_join.drop(index=NID_join.loc[NID_join.OtherStructureID.notnull()].index)


#A couple don't have Other Structure ID but are duplicates. Filter those by storage. Should only be 2.
# sort the data by descending max storage
NID_join = NID_join.sort_values('NID_Storag', ascending = False)

# Remove duplicate NIDs, keeping the first value aka the biggest capacity
bool_series = NID_join['NID'].duplicated()
NID_join = NID_join[~bool_series]


#Print checks
print('Size after removing duplicates and joining to site file:',NID_join.shape)
print('Number of sites in database:',NID_join.loc[NID_join.IsSite == 1].shape)
print('Number of Reclamation dams in database:', NID_join.loc[NID_join.IsUSBR == 1].shape)
print('Number of USACE dams in database:', NID_join.loc[NID_join.IsUSACE == 1].shape)

Size after removing duplicates and joining to site file: (90218, 64)
Number of sites in database: (1053, 64)
Number of Reclamation dams in database: (201, 64)
Number of USACE dams in database: (345, 64)


#### Update latitude and longitude based on known lat/long from the site file and populate fields.

In [9]:
NID_join = NID_join.reset_index()


# Set lat and long for sites to be the site lat/long
NID_join.loc[NID_join.IsSite == 1,'Latitude'] = NID_join.Lat
NID_join.loc[NID_join.IsSite == 1, 'Longitude'] = NID_join.Long

#Set lat and long for Reclamation dams
NID_join.loc[NID_join.IsUSBR == 1, 'Latitude'] = NID_join.Lat
NID_join.loc[NID_join.IsUSBR == 1, 'Longitude'] = NID_join.Long

NID_join.loc[NID_join.IsUSACE == 1, 'Latitude'] = NID_join.Lat
NID_join.loc[NID_join.IsUSACE == 1, 'Longitude'] = NID_join.Long


In [10]:
#Populate IsLock column
#replace null values with 0
NID_join.NumLocks = NID_join.NumLocks.fillna(0)
NID_join.LengthLocks = NID_join.LengthLocks.fillna(0)
NID_join.LockWidth = NID_join.LockWidth.fillna(0)
NID_join.LengthSecondLock = NID_join.LengthSecondLock.fillna(0)
NID_join.SecondLockWidth = NID_join.SecondLockWidth.fillna(0)
NID_join.loc[(NID_join.NumLocks>0) & (NID_join.NumLocks<10),'IsLock'] = 1
NID_join.loc[(NID_join.LengthLocks>0),'IsLock'] = 1
NID_join.loc[(NID_join.LockWidth>0),'IsLock'] = 1
NID_join.loc[(NID_join.LengthSecondLock>0),'IsLock'] = 1
NID_join.loc[(NID_join.SecondLockWidth>0),'IsLock'] = 1


#Search by name containing word Lock
NID_join['Dam_Name'] = NID_join['Dam_Name'].fillna('') #First fill NA names with empty strings
NID_join.loc[NID_join['Dam_Name'].str.contains('Lock '), 'IsLock'] = 1

#Set GA01804 and MO20537 to IsLock = 0 because are not locks but have lock in name
NID_join.loc[NID_join['NID'] == 'GA01804', 'IsLock'] = 0
NID_join.loc[NID_join['NID'] == 'MO20537', 'IsLock'] = 0

#Fill nans with 0
NID_join.IsLock = NID_join.IsLock.fillna(0)

#drop columns we no longer need
NID_join = NID_join.drop(['NumLocks','LengthLocks','LockWidth','LengthSecondLock','SecondLockWidth','NID_ID','Moved','Deleted','lat','long'],axis=1)


#### Filter dams by name to remove any dam names that contain Spillway, Levee, Sewage, Treatment, Auxiliary, or Remedial.

In [11]:
print('Size before filtering:', NID_join.shape)

## Filter by name: Filter the dams that are NOT sites
filters = "Spillway|Levee|Sewage|Treatment|Auxiliary|Remedial"

#Only filter non-sites
NIDs_filtered = NID_join.drop(index=NID_join.loc[NID_join.Dam_Name.str.contains(filters)==True].loc[(NID_join.IsSite==0)].loc[(NID_join.IsUSBR == 0)].loc[(NID_join.IsUSACE == 0)].index)

NID_join = NIDs_filtered

print('Size after filtering by name:',NIDs_filtered.shape)
print('Number of sites in database:',NID_join.loc[NID_join.IsSite == 1].shape) #this should be 535
print('Number of Reclamation dams in database:', NID_join.loc[NID_join.IsUSBR == 1].shape)
print('Number of USACE dams in database:', NID_join.loc[NID_join.IsUSACE == 1].shape)

Size before filtering: (90218, 55)
Size after filtering by name: (89734, 55)
Number of sites in database: (1053, 55)
Number of Reclamation dams in database: (201, 55)
Number of USACE dams in database: (345, 55)


#### Join remaining supplementary files and modify latitude and longitude appropriately.

In [12]:
## Join to GRanD dams

#Filter so only dams in the United States
GRanD = GRanD[GRanD['COUNTRY'].str.contains('United States')== True]
GRanD = GRanD[GRanD['ADMIN_UNIT'].str.contains('Alaska') == False]
GRanD = GRanD[GRanD['ADMIN_UNIT'].str.contains('Hawaii') == False]
GRanD = GRanD[GRanD['ADMIN_UNIT'].str.contains('Puerto Rico') == False]
                
#Filter out columns don't want
GRanD = GRanD.drop(columns = ['RIVER','ALT_RIVER','MAIN_BASIN','SUB_BASIN','NEAR_CITY','ALT_CITY','SEC_ADMIN','COUNTRY','SEC_CNTRY','ALT_YEAR','ALT_HGT_M','DAM_LEN_M','ALT_LEN_M','AREA_SKM','AREA_POLY','AREA_REP','AREA_MAX','AREA_MIN','CAP_MAX','CAP_REP','CAP_MIN','DEPTH_M','DIS_AVG_LS','DOR_PC','ELEV_MASL','CATCH_SKM','CATCH_REP','DATA_INFO','USE_IRRI','USE_ELEC','USE_SUPP','USE_FCON','USE_RECR','USE_NAVI','USE_FISH','USE_PCON','USE_LIVE','USE_OTHR','MAIN_USE','LAKE_CTRL','MULTI_DAMS','TIMELINE','COMMENTS','URL','QUALITY','EDITOR','POLY_SRC'])

print('Size of GRanD:', GRanD.shape)

Size of GRanD: (1897, 22)


In [13]:
#find matches between GRanD NID and NID NID; outer join should keep GRanD dams that weren't in NID
GRanD_join = pd.merge(NID_join, GRanD, on='NID', how='outer', suffixes = ('_NID','_GRanD'))

#combine output columns that were split in the join
GRanD_join.loc[GRanD_join.ShortID_GRanD.notnull(), 'ShortID'] = GRanD_join.ShortID_GRanD
GRanD_join.loc[GRanD_join.ShortID.isnull(), 'ShortID'] = GRanD_join.ShortID_NID

GRanD_join.loc[GRanD_join.IsSite_NID.isna(),'IsSite_NID'] = 0
GRanD_join['IsSite'] = GRanD_join['IsSite_NID']
GRanD_join.loc[GRanD_join.IsUSBR_NID.isna(),'IsUSBR_NID'] = 0
GRanD_join['IsUSBR'] = GRanD_join['IsUSBR_NID']
GRanD_join.loc[GRanD_join.IsUSACE_NID.isna(),'IsUSACE_NID'] = 0
GRanD_join['IsUSACE'] = GRanD_join['IsUSACE_NID']

#Create a GRanD lat/long field that takes the GRanD lat/long preferentially. These fields are called NewX and NewY for our manual placements
GRanD_join['LAT_GRAND'] = GRanD_join['NewY']
GRanD_join['LONG_GRAND'] = GRanD_join['NewX']

#set null values to -999
GRanD_join.LAT_GRAND = GRanD_join.LAT_GRAND.fillna(-999)
GRanD_join.LONG_GRAND = GRanD_join.LONG_GRAND.fillna(-999)


#Make GRanD lat/long null for sites; This preferentially chooses the site location over the GRanD location.
#If want to keep GRanD lat/long, skip this step
GRanD_join.loc[GRanD_join.IsSite == 1, 'LAT_GRAND'] = -999
GRanD_join.loc[GRanD_join.IsSite == 1, 'LONG_GRAND'] = -999


#Create a new field for lat/long that takes the GRanD lat/long if not null and takes the original lat/long if null
GRanD_join['LAT_JOIN'] = GRanD_join.LAT_GRAND
GRanD_join['LONG_JOIN'] = GRanD_join.LONG_GRAND
GRanD_join.loc[GRanD_join.LAT_GRAND == -999, 'LAT_JOIN'] = GRanD_join.Latitude
GRanD_join.loc[GRanD_join.LONG_GRAND == -999, 'LONG_JOIN'] = GRanD_join.Longitude

# #Create a column for capacity from GRanD
GRanD_join['GRanDCapm3'] = GRanD_join.CAP_MCM * (1e6) #million cubic meters to cubic m

print('Size before joining to GRanD:',NID_join.shape)
print('Size after joining to GRanD:',GRanD_join.shape) #should have 42 more
print('Number of sites in database:',GRanD_join.loc[GRanD_join.IsSite == 1].shape) 
print('Number of Reclamation dams in database:', GRanD_join.loc[GRanD_join.IsUSBR == 1].shape)
print('Number of USACE dams in database:', GRanD_join.loc[GRanD_join.IsUSACE == 1].shape)
print('Number of GRanD in database:', GRanD_join.loc[GRanD_join.IsGRanD == 1].shape) #this should be 1903

Size before joining to GRanD: (89734, 55)
Size after joining to GRanD: (89803, 85)
Number of sites in database: (1053, 85)
Number of Reclamation dams in database: (201, 85)
Number of USACE dams in database: (345, 85)
Number of GRanD in database: (1897, 85)


In [14]:
#find matches between Removed Dams and NID NID; outer join should keep Removed dams that weren't in NID. If you do not want to use a removed dams file, skip this step.
GRanD_join = pd.merge(GRanD_join, removed, on='NID', how='outer', suffixes = ('_join','_rem'))


# #Create a GRanD lat/long field that takes the maximum lat/long out of the two joins
GRanD_join['LAT_Rem'] = GRanD_join['DamLatitud']
GRanD_join['LONG_Rem'] = GRanD_join['DamLongitu']

#set null values to -999
GRanD_join.LAT_Rem = GRanD_join.LAT_Rem.fillna(-999)
GRanD_join.LONG_Rem = GRanD_join.LONG_Rem.fillna(-999)

#Make GRanD lat/long null for sites, GRanD, USBR, and USACE. This preferentially keeps their locations.
GRanD_join.loc[GRanD_join.IsSite == 1, 'LAT_Rem'] = -999
GRanD_join.loc[GRanD_join.IsSite == 1, 'LONG_Rem'] = -999

GRanD_join.loc[GRanD_join.IsGRanD == 1, 'LAT_Rem'] = -999
GRanD_join.loc[GRanD_join.IsGRanD == 1, 'LONG_Rem'] = -999                                                                                    
                                                                                    
GRanD_join.loc[GRanD_join.IsUSBR == 1, 'LAT_Rem'] = -999
GRanD_join.loc[GRanD_join.IsUSBR == 1, 'LONG_Rem'] = -999

GRanD_join.loc[GRanD_join.IsUSACE == 1, 'LAT_Rem'] = -999
GRanD_join.loc[GRanD_join.IsUSACE == 1, 'LONG_Rem'] = -999
                                                                                    
#Create a new field for lat/long that takes the Removed dam lat/long if not null and takes the original lat/long if null
GRanD_join.loc[GRanD_join.LAT_Rem != -999, 'LAT_JOIN'] = GRanD_join.LAT_Rem
GRanD_join.loc[GRanD_join.LONG_Rem != -999, 'LONG_JOIN'] = GRanD_join.LONG_Rem


# #Fix duplicate fields from join
GRanD_join.loc[GRanD_join.Batch_for_rem.isnull() == True,'Batch_for_rem'] = -999 #set null values to -999
GRanD_join.loc[GRanD_join.Batch_for_rem != -999, 'Batch_for'] = GRanD_join.Batch_for_rem
GRanD_join.loc[GRanD_join.Batch_for_rem == -999, 'Batch_for'] = GRanD_join.Batch_for_join

print('Number of dams in database after joining to removed dams:',GRanD_join.shape)



Number of dams in database after joining to removed dams: (89846, 109)


In [15]:
#join to GeoDAR and add GeoDAR locations where able
NID_geoDAR = pd.merge(GRanD_join,geoDAR[['NID','NewX','NewY']],on='NID', how='left')

# #Create a GRanD lat/long field that takes the maximum lat/long out of the two joins
NID_geoDAR['LAT_Geo'] = NID_geoDAR['NewY_y']
NID_geoDAR['LONG_Geo'] = NID_geoDAR['NewX_y']

#set null values to -999
NID_geoDAR.LAT_Geo = NID_geoDAR.LAT_Geo.fillna(-999)
NID_geoDAR.LONG_Geo = NID_geoDAR.LONG_Geo.fillna(-999)

#Make GRanD lat/long null for sites, GRanD, USACE, and USBR. This preferentially keeps their locations.
NID_geoDAR.loc[NID_geoDAR.IsSite == 1, 'LAT_Geo'] = -999
NID_geoDAR.loc[NID_geoDAR.IsSite == 1, 'LONG_Geo'] = -999

NID_geoDAR.loc[NID_geoDAR.IsGRanD == 1, 'LAT_Geo'] = -999
NID_geoDAR.loc[NID_geoDAR.IsGRanD == 1, 'LONG_Geo'] = -999                                                                                    
                                                                                    
NID_geoDAR.loc[NID_geoDAR.IsUSBR == 1, 'LAT_Geo'] = -999
NID_geoDAR.loc[NID_geoDAR.IsUSBR == 1, 'LONG_Geo'] = -999

NID_geoDAR.loc[NID_geoDAR.IsUSACE == 1, 'LAT_Geo'] = -999
NID_geoDAR.loc[NID_geoDAR.IsUSACE == 1, 'LONG_Geo'] = -999
                                                                                    
#Create a new field for lat/long that takes the GeoDAR lat/long if not null and takes the original lat/long if null
NID_geoDAR.loc[NID_geoDAR.LAT_Rem != -999, 'LAT_JOIN'] = NID_geoDAR.LAT_Geo
NID_geoDAR.loc[NID_geoDAR.LONG_Rem != -999, 'LONG_JOIN'] = NID_geoDAR.LONG_Geo

print('Number of dams in database:', NID_geoDAR.shape)

GRanD_join = NID_geoDAR

Number of dams in database: (89846, 113)


#### Filter based on storage

In [16]:
#assign 0 to IsGRanD non-GRanD dams
GRanD_join.IsGRanD = GRanD_join.IsGRanD.fillna(0)

## Filter based on storage
print('Size before storage filtering:',GRanD_join.shape)

#Create column that takes the maximum of all of the storage values. This is NID_Storag in the NID table
GRanD_join['MaxStor_m3'] = GRanD_join['NID_Storag'] * 1233.48 #Convert AF to m^3

#Fill MaxStor nans with 0
GRanD_join.MaxStor_m3 = GRanD_join.MaxStor_m3.fillna(0)


#Set GRanD_join storage sources to initially be NID and reference year to initially be Year_Compl
#The outcome of this is that for each storage value reported, you have a source and a year that storage value represents
GRanD_join['StorSource'] = 'NID'
GRanD_join['Stor_Refyr'] = GRanD_join.Year_Compl

#Replace any with MaxStor == 0 with GRanD storage, site storage, then USBR/USACE storage, then removed dams
GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'StorSource'] = 'GRanD'
GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'Stor_Refyr'] = GRanD_join.YEAR
GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'MaxStor_m3'] = GRanD_join.GRanDCapm3 #GRanD

GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'StorSource'] = GRanD_join.Batch_for
GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'Stor_Refyr'] = GRanD_join.Year_First
GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'MaxStor_m3'] = GRanD_join.CapOrigAF*1233.48 #site

GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'StorSource'] = GRanD_join.Batch_for
GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'Stor_Refyr'] = GRanD_join.yr_p
GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'MaxStor_m3'] = GRanD_join.Capm3_p #Reclamation/USACE

GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'StorSource'] = GRanD_join.Batch_for
GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'Stor_Refyr'] = GRanD_join.yrc
GRanD_join.loc[GRanD_join.MaxStor_m3 == 0, 'MaxStor_AF'] = GRanD_join.OrigCap_m3 #Removed dams


#Any dams with max-storage = 0 that is not a site or federal is removed
GRanD_join = GRanD_join.drop(index=GRanD_join.loc[GRanD_join.MaxStor_m3 == 0].loc[GRanD_join.IsSite==0 & (GRanD_join.IsUSBR == 0) & (GRanD_join.IsUSACE==0)].index)

print('Size after storage filtering:',GRanD_join.shape)
print('Number of sites in database:', GRanD_join.loc[GRanD_join.IsSite == 1].shape)
print('Number of Reclamation dams in database:', GRanD_join.loc[GRanD_join.IsUSBR == 1].shape)
print('Number of USACE dams in database:', GRanD_join.loc[GRanD_join.IsUSACE == 1].shape)
print('Number of GRanD in database:', GRanD_join.loc[GRanD_join.IsGRanD == 1].shape)

Size before storage filtering: (89846, 113)
Size after storage filtering: (89846, 117)
Number of sites in database: (1053, 117)
Number of Reclamation dams in database: (201, 117)
Number of USACE dams in database: (345, 117)
Number of GRanD in database: (1897, 117)


#### Assign ShortIDs to any dam that doesn't already have one.

In [17]:
## Assign ShortID to non-sites
# First, sort by ascending NID
GRanD_join = GRanD_join.sort_values('NID',ascending = True)


#starting ShortID should be the maximum of the site/GRanD ShortIDs plus 1,000 and rounded to the nearest thousandth
startID = math.floor((GRanD_join.ShortID.max() + 1000)/1000)*1000
ID = startID

#Assign a ShortID to anything that doesn't have one yet.        
for index, row in GRanD_join.iterrows():
    if pd.isna(row['ShortID']):  # Check if ShortID is null
        GRanD_join.loc[index, 'ShortID'] = ID
        ID += 1  # Increment ID for next ShortID


In [18]:
#check for duplicate ShortIDs. If there are duplicates, you have an error in your site input files. This is either
#you assigned two dams with different NIDs the same ShortID or could have a typo in the NID field.
test = GRanD_join.ShortID

nodup = set(test)

if len(nodup) != len(test):
    print('There are duplicate ShortIDs!')

    newlist = [] # empty list to hold unique elements from the list
    duplist = [] # empty list to hold the duplicate elements from the list
    for i in test:
        if i not in newlist:
            newlist.append(i)
        else:
            duplist.append(i) # this method catches the first duplicate entries, and appends them to the list
            
    # The next step is to print the duplicate entries, and the unique entries
    print("List of duplicates", duplist)
else:
    print('There are no duplicate ShortIDs')


There are no duplicate ShortIDs


#### Clean up columns and export as a csv

In [19]:
#Combine columns where necessary and drop unwanted columns

#Combine NID Dam height and removed dam heights into one field. Only put in removed dam where NID is null.
GRanD_join['DamH_ft'] = GRanD_join.NID_Height
GRanD_join.loc[(GRanD_join.DamH_ft.isnull() == True),'DamH_ft'] = GRanD_join.DAmHft
GRanD_join.loc[(GRanD_join.DamH_ft.isnull())==True,'DamH_ft'] = GRanD_join.DAM_HGT_M*3.28

#Fill in Dam function from removed dam file everywhere PrimPurp is null.Then replace the null PrimaryPurp with Purp because some have null PrimaryPurp and non-null Purp (looking at you random person in SD)
GRanD_join.loc[(GRanD_join.PrimaryPurp.isnull() == True),'PrimaryPurp'] = GRanD_join.DamFunctio
GRanD_join.loc[(GRanD_join.PrimaryPurp.isnull() == True),'PrimaryPurp'] = GRanD_join.Purp

#year completed
GRanD_join.loc[GRanD_join.yrc.isnull()==True, 'yrc'] = GRanD_join.Year_Compl #anywhere removed dam database is null, change to NID
GRanD_join.loc[GRanD_join.IsSite == 1, 'yrc'] = GRanD_join.Year_Compl_site
GRanD_join.loc[GRanD_join.IsUSBR == 1, 'yrc'] = GRanD_join.Year_Compl_site
GRanD_join.loc[GRanD_join.IsUSACE == 1, 'yrc'] = GRanD_join.Year_Compl_site
GRanD_join.loc[GRanD_join.yrc.isnull()==True, 'yrc'] = GRanD_join.YEAR #anything else still null fill with GRanD

#year removed
GRanD_join.loc[GRanD_join.yrr.isnull()==True, 'yrr'] = GRanD_join.YrRemoved #site/USACE/USBR
GRanD_join.loc[GRanD_join.yrr.isnull()==True, 'yrr'] = GRanD_join.REM_YEAR #GRanD

#dam name
GRanD_join.loc[GRanD_join.Dam_Name=='','Dam_Name'] = np.nan #set values we made blank earlier back to nan
GRanD_join.loc[GRanD_join.Dam_Name.isnull()==True, 'Dam_Name'] = GRanD_join.Other_Dam #Replace missing NID names with NID other name first
GRanD_join.loc[GRanD_join.Dam_Name.isnull()==True, 'Dam_Name'] = GRanD_join.Reservoir #then site
GRanD_join.loc[GRanD_join.Dam_Name.isnull()==True, 'Dam_Name'] = GRanD_join.USBRname #then USBR
GRanD_join.loc[GRanD_join.Dam_Name.isnull()==True, 'Dam_Name'] = GRanD_join.RES_NAME #then GRanD
GRanD_join.loc[GRanD_join.Dam_Name.isnull()==True, 'Dam_Name'] = GRanD_join.DamName #then Removed file

#Drop unwanted columns at this point.
GRanD_join = GRanD_join.drop(['index','OtherStructureID','FederalID','Longitude','Latitude','River','Owner_Name',
                             'Max_Storag','Normal_Sto','Lat','Long','Batch_for_join','ShortID_NID',
                              'IsSite_NID','IsUSBR_NID','IsUSACE_NID','DAM_NAME','ALT_NAME',
                              'ADMIN_UNIT','LONG_DD','LAT_DD','NewX_x','NewY_x','NIDnotes','ShortID_GRanD','HasNHD',
                              'IsSite_GRanD','IsUSBR_GRanD','IsUSACE_GRanD','Field','LAT_GRAND','LONG_GRAND',
                              'CitationID','CitationUR','DamAccessi','DamRiverNa','DamRiver_1','DamLocatio',
                              'DamState_P','DamLatitud','DamLongitu','DamAccurac','DamOwner','Batch_for_rem',
                              'LAT_Rem','LONG_Rem','NewX_y','NewY_y','LAT_Geo','LONG_Geo','NID_Height','DamH_m',
                              'DAmHft','Purp','DamFunctio','SiteIsGRanD','GRanD_ID','Other_Dam','REM_YEAR','RES_NAME',
                              'DamName','DAM_HGT_M','USACE_PROJECT_ID','YEAR'], axis=1)


C:\Users\ahurst\AppData\Local\Temp\1\ipykernel_33772\2525771764.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  GRanD_join.loc[GRanD_join.yrr.isnull()==True, 'yrr'] = GRanD_join.YrRemoved #site/USACE/USBR


In [20]:
#Convert everything to metric and drop the ft column
GRanD_join['Dam_Len_m'] = GRanD_join.Dam_Length*0.3048
GRanD_join['SA_m2'] = GRanD_join.Surface_Ar*4046.85642
GRanD_join['DA_km2'] = GRanD_join.Drainage_A*2.58998811
GRanD_join['MaxQ_m3s'] = GRanD_join.Max_Discha*0.028316847
GRanD_join['DamH_m'] = GRanD_join.DamH_ft * 0.3048
GRanD_join['NIDStor_m3'] = GRanD_join.NID_Storag*1233.48185532
GRanD_join['GRanDCapm3'] = GRanD_join.CAP_MCM*(1e6)
GRanD_join['elev_m'] = GRanD_join.elev_ft*0.3048


GRanD_join = GRanD_join.drop(['Dam_Length','Surface_Ar','Drainage_A','Max_Discha','CapOrigAF','CapNewAF','CapAF_p',
                             'DamNameAlt','NID_Storag','CAP_MCM','elev_ft','DamH_ft',
                              ],axis=1)
#Rename columns
GRanD_join.rename(columns = {'OrigCap_m3':'OCapm3_Rem','LAT_JOIN':'LAT_FINAL','LONG_JOIN':'LONG_FINAL',
                             'DA_km':'site_DA_km'}, inplace=True)

In [21]:
#Export final files as a csv
GRanD.to_csv(os.path.join(out_folder,'GRanD.csv'))
GRanD_join.to_csv(os.path.join(out_folder,'NID_GRanDjoin.csv'))



# 2. Snap dams to NHDPlus Flowlines.

In [22]:
## Snap dams to NHDPlus HR Flowlines: must be done with arcPy. Ensure that all layers are in the same coordinate system (here we use NAD83)

#First convert the csv to a shapefile
XFieldName = 'LONG_FINAL'
YFieldName = 'LAT_FINAL'
newLayerName = "NID_filtered" #Name of your output shapefile

spatialRef = arcpy.SpatialReference(4269) #spatial reference WKID for NAD83
csvFilePath = os.path.join(out_folder,'NID_GRanDjoin.csv') #your filtered dam dataset csv

arcpy.MakeXYEventLayer_management(csvFilePath, XFieldName, YFieldName, newLayerName, spatial_reference=spatialRef)
arcpy.FeatureClassToShapefile_conversion(newLayerName, out_folder)


<Result 'D:\\ResSed\\MediumResolution_DamLinkages\\Manuscript\\Outputs'>

In [23]:
#For NHDPlus
NIDFiltered = os.path.join(out_folder,'NID_filtered.shp') #Link to your filtered NID shapefile

NIDlyr = "NIDlyr" #create a layer file
NHDlyr = "NHDlyr"
arcpy.management.MakeFeatureLayer(NIDFiltered,NIDlyr) #convert the feature class to a layer to work from
arcpy.management.MakeFeatureLayer(NHDFlowline,NHDlyr)

#Run the near tool to get the new lat/long with near FType558
arcpy.management.SelectLayerByAttribute(NHDlyr,'NEW_SELECTION',"FCODE = 55800")

arcpy.management.SelectLayerByAttribute(NIDlyr,'NEW_SELECTION',"IsSite = 0") #select all non-sites, non-Reclamation, non-USACE, and non-GRanD to snap
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsGRanD = 0")
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsUSBR = 0")
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsUSACE = 0")

#Near 250 m to FType558 for everything that isn't a site
arcpy.analysis.Near(NIDlyr,NHDlyr,"250 Meters","LOCATION","NO_ANGLE","PLANAR")

#Create a new field and populate it
arcpy.management.AddField(NIDlyr,'NrX_Final',"DOUBLE")
arcpy.management.AddField(NIDlyr,'NrY_Final',"DOUBLE")


arcpy.management.SelectLayerByAttribute(NHDlyr,'CLEAR_SELECTION') #clear the flowline FType 558 Selection before transferring over values

#Transfer values over
arcpy.CalculateField_management(NIDlyr, "NrX_Final","!NEAR_X!", "PYTHON3")
arcpy.CalculateField_management(NIDlyr, "NrY_Final","!NEAR_Y!", "PYTHON3")

arcpy.management.DeleteField(NIDlyr,['NEAR_X','NEAR_Y','NEAR_DIST','NEAR_FID'])

#Near 500m to all flowlines for any that didn't snap and have MaxStor >= 4000 AF/5,0000,000 m^3
arcpy.management.SelectLayerByAttribute(NHDlyr,'NEW_SELECTION',"FCODE <> 56600") #select NHD that isn't a coastline
arcpy.management.SelectLayerByAttribute(NIDlyr,'NEW_SELECTION',"NrX_Final = -1") #Select any Dams that didn't snap
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsSite = 0") #And any that aren't sites
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsGRanD = 0") #And not GRanD
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsUSBR = 0") #And not Reclamation
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsUSACE = 0") #And not Army Corps
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"MaxStor_m3>=5e6") #MaxStor >= 4000 AF

arcpy.analysis.Near(NIDlyr,NHDlyr,"500 Meters","LOCATION","NO_ANGLE","PLANAR")

#Transfer values over
arcpy.CalculateField_management(NIDlyr, "NrX_Final","!NEAR_X!", "PYTHON3")
arcpy.CalculateField_management(NIDlyr, "NrY_Final","!NEAR_Y!", "PYTHON3")

arcpy.management.DeleteField(NIDlyr,['NEAR_X','NEAR_Y','NEAR_DIST','NEAR_FID'])

#Near 250m to all flowlines for any that didn't snap and still aren't a site
arcpy.management.SelectLayerByAttribute(NHDlyr,'NEW_SELECTION',"FCODE <> 56600") #select NHD that isn't a coastline
arcpy.management.SelectLayerByAttribute(NIDlyr,'NEW_SELECTION',"NrX_Final = -1") #Select any Dams that didn't snap
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsSite = 0") #And any that aren't sites
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsGRanD = 0") #And any that aren't GRanD
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsUSBR = 0") #And not Reclamation
arcpy.management.SelectLayerByAttribute(NIDlyr,'SUBSET_SELECTION',"IsUSACE = 0") #And not Army Corps


arcpy.analysis.Near(NIDlyr,NHDlyr,"250 Meters","LOCATION","NO_ANGLE","PLANAR")

#Transfer values over
arcpy.CalculateField_management(NIDlyr, "NrX_Final","!NEAR_X!", "PYTHON3")
arcpy.CalculateField_management(NIDlyr, "NrY_Final","!NEAR_Y!", "PYTHON3")

arcpy.management.DeleteField(NIDlyr,['NEAR_X','NEAR_Y','NEAR_DIST','NEAR_FID'])

#Sites to nearest flowline using near 250m b/c should be in the correct place.
arcpy.management.SelectLayerByAttribute(NIDlyr,'CLEAR_SELECTION') #clear the NID selection
arcpy.management.SelectLayerByAttribute(NIDlyr,'NEW_SELECTION',"IsSite = 1") #Select all sites

arcpy.analysis.Near(NIDlyr,NHDlyr,"250 Meters","LOCATION","NO_ANGLE","PLANAR")

#Transfer values over
arcpy.CalculateField_management(NIDlyr, "NrX_Final","!NEAR_X!", "PYTHON3")
arcpy.CalculateField_management(NIDlyr, "NrY_Final","!NEAR_Y!", "PYTHON3")

arcpy.management.DeleteField(NIDlyr,['NEAR_X','NEAR_Y','NEAR_DIST','NEAR_FID'])


#GRanD to nearest flowline using near 250m b/c should be in the correct place.
arcpy.management.SelectLayerByAttribute(NIDlyr,'CLEAR_SELECTION') #clear the NID selection
arcpy.management.SelectLayerByAttribute(NIDlyr,'NEW_SELECTION',"IsGRanD = 1") #Select all GRanD

arcpy.analysis.Near(NIDlyr,NHDlyr,"250 Meters","LOCATION","NO_ANGLE","PLANAR")

#Transfer values over
arcpy.CalculateField_management(NIDlyr, "NrX_Final","!NEAR_X!", "PYTHON3")
arcpy.CalculateField_management(NIDlyr, "NrY_Final","!NEAR_Y!", "PYTHON3")

arcpy.management.DeleteField(NIDlyr,['NEAR_X','NEAR_Y','NEAR_DIST','NEAR_FID'])


#USBR to nearest flowline using near 250m b/c should be in the correct place.
arcpy.management.SelectLayerByAttribute(NIDlyr,'CLEAR_SELECTION') #clear the NID selection
arcpy.management.SelectLayerByAttribute(NIDlyr,'NEW_SELECTION',"IsUSBR = 1") #Select all USBR

arcpy.analysis.Near(NIDlyr,NHDlyr,"250 Meters","LOCATION","NO_ANGLE","PLANAR")

#Transfer values over
arcpy.CalculateField_management(NIDlyr, "NrX_Final","!NEAR_X!", "PYTHON3")
arcpy.CalculateField_management(NIDlyr, "NrY_Final","!NEAR_Y!", "PYTHON3")

arcpy.management.DeleteField(NIDlyr,['NEAR_X','NEAR_Y','NEAR_DIST','NEAR_FID'])


#USACE to nearest flowline using near 250m b/c should be in the correct place.
arcpy.management.SelectLayerByAttribute(NIDlyr,'CLEAR_SELECTION') #clear the NID selection
arcpy.management.SelectLayerByAttribute(NIDlyr,'NEW_SELECTION',"IsUSACE = 1") #Select all USACE

arcpy.analysis.Near(NIDlyr,NHDlyr,"250 Meters","LOCATION","NO_ANGLE","PLANAR")

#Transfer values over
arcpy.CalculateField_management(NIDlyr, "NrX_Final","!NEAR_X!", "PYTHON3")
arcpy.CalculateField_management(NIDlyr, "NrY_Final","!NEAR_Y!", "PYTHON3")

arcpy.management.DeleteField(NIDlyr,['NEAR_X','NEAR_Y','NEAR_DIST','NEAR_FID'])


#Delete any features that did not snap.
#First select non-snaps
arcpy.management.SelectLayerByAttribute(NIDlyr,'CLEAR_SELECTION')
arcpy.management.SelectLayerByAttribute(NIDlyr,'NEW_SELECTION',"NrX_Final = -1")

#Delete the selected rows:
arcpy.management.DeleteRows(NIDlyr)
arcpy.GetCount_management(NIDlyr)

#Display XY data using NearX and NearY to move the points onto the NHD Flowlines
arcpy.management.MakeXYEventLayer(NIDlyr, "NrX_Final", "NrY_Final", 'NIDFiltered_snap', spatialRef)


<Result 'NIDFiltered_snap'>

### Intersect with the flowline data to extract attributes.

In [24]:
#Intersect file with moved dams to the flowline file:
arcpy.analysis.Intersect(['NIDFiltered_snap',NHDFlowline],os.path.join(out_folder,'NIDFiltered_snap.shp'),"ALL",None,"INPUT")

#Export NID file
dbf = Dbf5(os.path.join(out_folder,'NIDFiltered_snap.dbf'))
df = dbf.to_dataframe()
df.to_csv(os.path.join(out_folder,'NID_filtered_snapped.csv'),index=False)

### Remove duplicate values.

In [25]:
#Remove duplicates. When you run intersect in ArcGIS, any intersects that happen at the join of two lines gives two
#results in the final table. We need to delete one of these.

NID = pd.read_csv(os.path.join(out_folder,'NID_filtered_snapped.csv')) #load data
NID_sort = NID.sort_values('ShortID', ascending = True) #sort by ascending ShortID

print('Size before duplicates removed:',NID_sort.shape)

#convert DF to dictionaries (struct-like); basically has format column->value
NID_dict = NID_sort.to_dict(orient = 'records')
dupl_ordered_dict = NID.to_dict(orient='records')

#initialize empty list to store indices of non-duplicates
dupind = []

#Identify unique values and their counts
shortID = [item['ShortID'] for item in dupl_ordered_dict]
uniquevals,ia = np.unique(shortID, return_inverse = True)

#Count the frequency of each index in ia
bincounts = np.bincount(ia)

#Zero out singles
singles = uniquevals[bincounts <= 1]
singleidx = [i for i, val in enumerate(shortID) if val in singles]
for idx in singleidx:
    shortID[idx] = 0
    
#Overwrite repeats
repeats = uniquevals[bincounts > 1]
shortID = np.array([np.where(repeats == val)[0][0] + 1 if val in repeats else val for val in shortID])

C:\Users\ahurst\AppData\Local\Temp\1\ipykernel_33772\1637887619.py:4: DtypeWarning: Columns (25,188) have mixed types. Specify dtype option on import or set low_memory=False.
  NID = pd.read_csv(os.path.join(out_folder,'NID_filtered_snapped.csv')) #load data


Size before duplicates removed: (75990, 192)


In [ ]:
skip_it = 0; #initialize a counter
#Pull out FCODE and Hydrosequence fields to help with decision tree for removing duplicates
FCODE = [item['FCODE'] for item in dupl_ordered_dict]
Hydroseq = [item['Hydroseq'] for item in dupl_ordered_dict]


for i in range(len(shortID)):
    if shortID[i] == 0: #if it is not a duplicate, keep it
        dupind.append(i)
    elif skip_it>0: #or if we already dealt with it, update the counter so it gets skipped
        skip_it -= 1
        continue
    else: #else the value is a duplicate
        dup = [idx for idx, val in enumerate(shortID) if val == shortID[i]] #gives all indices of the duplicates
        dup1 = dup[0]
        dupskip = dup1 #keep track of what the first index was because we will change this
        j = len(dup)
        jskip = j #same for the length of the duplicates

        Hydro = Hydroseq[dup1:dup[j-1]+1] #Pull out Hydrosequences as the duplicates

        kept_indices = [i for i, x in enumerate(Hydro) if x not in [Hydroseq[i] for i in dupind]] #if a dam is already snapped to that flowline, remove the flowlines from the options to choose from

        dup_test = [dup[i] for i in kept_indices]
                
        if len(dup_test) == 0: #all of the flowline options have already been used, in which case just keep them all. duplicate snaps are removed later.
            dup_test = dup
        
        dup = dup_test
        
        dup1 = dup[0]
        j = len(dup)
        
        Floc = [index for index, value in enumerate(FCODE[dup1:dup[j-1]+1]) if value == 55800] #pull out FCODE = 55800 for duplicates
        coast = [index for index, value in enumerate(FCODE[dup1:dup[j-1]+1]) if value == 56600] #pull out any duplicates that are on a coast flowline
        
        
    
        if len(Floc) == 1: #If only one value is FType 558
            dupind.append(dup[Floc[0]])
        elif len(Floc) == j: #all of the values are 558, take smallest hydroseq (most downstream)
            Hydro = Hydroseq[dup1:dup[j-1]+1]
            minloc = np.argmin(Hydro)
            dupind.append(dup[minloc])
        elif len(Floc) == 0: #none are FType 558         
            Hydro = Hydroseq[dup1:dup[j-1]+1]
           
            if len(coast)>0:
                Hydro = np.delete(Hydro,coast) #remove coastal values; any dam that snaps to a coast flowline has it's dam order messed up and can route along the coast
                dup = np.delete(dup,coast)

            minloc = np.argmin(Hydro) #currently taking minimum of the new hydro
            dupind.append(dup[minloc])
        else: #some other number of values is FType 558; still take the most downstream
            Hydro = [Hydroseq[dup[index]] for index in Floc]
    
            if len(coast) > 0:
                Hydro = np.delete(Hydro,coast)
                dup = np.delete(dup,coast)
                
            minloc = np.argmin(Hydro)
            dupind.append(dup[Floc[minloc]])
            
        if dupskip == i: #if the first index was the current index
            skip_it = jskip-1 #skip the next j-1 indices
        else:
            skip_it = 0

dupltable = pd.DataFrame.from_dict(dupl_ordered_dict)
noduplicates = dupltable.loc[dupind]

noduplicates.to_csv(os.path.join(out_folder,'NID_filtered_snapped_nodupl.csv'),index = False)


print('Size after removing duplicates:',noduplicates.shape)


# 3. Join D50 Data

In [ ]:
NID_D50 = pd.merge(noduplicates, D50, on='COMID', how='left')
NID_D50 = NID_D50.drop(['Shape_Leng','OID_','StreamOrde_y','TotDASqKM_y'],axis=1)
NID_D50.rename(columns = {"TotDASqKM_x":"TotDASqKM","StreamOrde_x":"StreamOrder"},inplace=True)

print('Size after joining D50:',NID_D50.shape)

NID_D50.to_csv(os.path.join(out_folder,'NID_filtered_snapped_nodupl_D50.csv'),index = False)